# Champlin Smith Project Milestone 2

In [53]:
import pandas as pd


file_path = "C:\\Users\\champ\\OneDrive\\Documents\\MS Data Science\\DSC550 Data Mining\\project\\train.csv"
df = pd.read_csv(file_path, sep=',', encoding='ISO-8859-1')

print(df.head(10))

       ID Customer_ID     Month             Name   Age          SSN  \
0  0x1602   CUS_0xd40   January    Aaron Maashoh    23  821-00-0265   
1  0x1603   CUS_0xd40  February    Aaron Maashoh    23  821-00-0265   
2  0x1604   CUS_0xd40     March    Aaron Maashoh  -500  821-00-0265   
3  0x1605   CUS_0xd40     April    Aaron Maashoh    23  821-00-0265   
4  0x1606   CUS_0xd40       May    Aaron Maashoh    23  821-00-0265   
5  0x1607   CUS_0xd40      June    Aaron Maashoh    23  821-00-0265   
6  0x1608   CUS_0xd40      July    Aaron Maashoh    23  821-00-0265   
7  0x1609   CUS_0xd40    August              NaN    23    #F%$D@*&8   
8  0x160e  CUS_0x21b1   January  Rick Rothackerj   28_  004-07-5839   
9  0x160f  CUS_0x21b1  February  Rick Rothackerj    28  004-07-5839   

  Occupation Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  \
0  Scientist      19114.12            1824.843333                  3  ...   
1  Scientist      19114.12                    NaN               

C:\Users\champ\AppData\Local\Temp\ipykernel_33312\3130938201.py:5: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',', encoding='ISO-8859-1')


In [54]:
# Filter out invalid ages
df['Age'] = pd.to_numeric(df['Age'], errors='coerce').astype('float')
df = df[(df['Age'] >= 0) & (df['Age'] <= 115)]
print(df.head(10))

        ID Customer_ID     Month             Name   Age          SSN  \
0   0x1602   CUS_0xd40   January    Aaron Maashoh  23.0  821-00-0265   
1   0x1603   CUS_0xd40  February    Aaron Maashoh  23.0  821-00-0265   
3   0x1605   CUS_0xd40     April    Aaron Maashoh  23.0  821-00-0265   
4   0x1606   CUS_0xd40       May    Aaron Maashoh  23.0  821-00-0265   
5   0x1607   CUS_0xd40      June    Aaron Maashoh  23.0  821-00-0265   
6   0x1608   CUS_0xd40      July    Aaron Maashoh  23.0  821-00-0265   
7   0x1609   CUS_0xd40    August              NaN  23.0    #F%$D@*&8   
9   0x160f  CUS_0x21b1  February  Rick Rothackerj  28.0  004-07-5839   
10  0x1610  CUS_0x21b1     March  Rick Rothackerj  28.0  004-07-5839   
11  0x1611  CUS_0x21b1     April  Rick Rothackerj  28.0  004-07-5839   

   Occupation Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  \
0   Scientist      19114.12            1824.843333                  3  ...   
1   Scientist      19114.12                    NaN 

In [55]:
# Dropping PII fields - not needed as we have a unique identifier to derive these fields if needed in future
df = df.drop(columns=['Name', 'SSN'])

In [56]:
# Clean the Annual Income column
df = df[~df['Annual_Income'].astype(str).str.endswith('_')]
df['Annual_Income'] = pd.to_numeric(df['Annual_Income'], errors='coerce')

In [57]:
# Get the Median Annual Income for the person's occupation, and then get the difference between their income and their occupation's median.
# This will show is where they are in the occupation's "Salary Band" and might give us info on if they are high performing or not which might impact credit score.
median_income_by_occ = df.groupby('Occupation')['Annual_Income'].median().rename('Median Occupation Income')
df = df.join(median_income_by_occ, on='Occupation')
df['Income vs Occupation Median'] = df['Annual_Income'] - df['Median Occupation Income']

print(df.head(10))

        ID Customer_ID     Month   Age Occupation  Annual_Income  \
0   0x1602   CUS_0xd40   January  23.0  Scientist       19114.12   
1   0x1603   CUS_0xd40  February  23.0  Scientist       19114.12   
3   0x1605   CUS_0xd40     April  23.0  Scientist       19114.12   
4   0x1606   CUS_0xd40       May  23.0  Scientist       19114.12   
5   0x1607   CUS_0xd40      June  23.0  Scientist       19114.12   
6   0x1608   CUS_0xd40      July  23.0  Scientist       19114.12   
7   0x1609   CUS_0xd40    August  23.0  Scientist       19114.12   
9   0x160f  CUS_0x21b1  February  28.0    Teacher       34847.84   
11  0x1611  CUS_0x21b1     April  28.0    Teacher       34847.84   
12  0x1612  CUS_0x21b1       May  28.0    Teacher       34847.84   

    Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  \
0             1824.843333                  3                4              3   
1                     NaN                  3                4              3   
3          

In [58]:
# Create dummy variables for occupation
df = pd.get_dummies(df, columns=['Occupation'], prefix='Occ')

In [59]:
# Clean Interest rate outliers
df = df[df['Interest_Rate'] <= 50]

In [60]:
# Clean Number of credit card outliers
df = df[df['Num_Credit_Card'] <= 30]

In [61]:
# Clean Number of loan outliers
df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce').astype('float')
df = df[df['Num_of_Loan'] <= 15]